In [245]:
import numpy

In [246]:
# Пока вспомогательный код, алгоритм начинается ниже

class Matrix:
    """Чтобы использовать индексы, начиная с 1, а в нулевом столбце хранить свободные члены"""
    def __getitem__(self, index):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1]
            return self.m[i, j]
        else:
            return self.m[index - 1]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1]
            self.m[i, j] = value
        else:
            self.m[index - 1] = value

    def __init__(self, n):
        self.n = n
        self.m = numpy.zeros((n, n + 1))



    def lambdamax(self):
        """Максимальное по модулю собственное число"""
        A = self.m[:,1:] #Выкидывает первый столбец, чтобы получить квадратную матрицу
        x_old = numpy.random.rand(self.n)
        norm_old = numpy.linalg.norm(x_old)
        x_old /= norm_old
        #x_new = A.dot(x_old)

        success = False
        for _ in range(1000):
            x_new = A.dot(x_old)
            norm_new = numpy.linalg.norm(x_new)
            x_new /= norm_new
            if numpy.dot(x_old, x_new) > 0.999 and abs(norm_new/norm_old - 1) < 0.001 :
                success = True
                break
            x_old = x_new
            norm_old = norm_new

        if not success:
            print("failure")
        return norm_new

    def norm(self):
        """1 норма матрицы - максимальная сумма модулей в строке"""
        sumofmodulesarray = []

        for i in range(1, self.n + 1):
            sumofmodules = 0

            for j in range(1, self.n + 1):
                sumofmodules += abs(self[i, j])
            sumofmodulesarray.append(sumofmodules)

        return max(sumofmodulesarray)


    def dot(self, x):
        """Умножает матрицу на вектор с учётом съехавших индексов"""
        x_ = [0] + x
        return self.m.dot(x_)

In [247]:
class GaussMatrix:
    """Расширенная матрица. Нулевой столбец - для свободных членов, положительные - для матрицы коэффициентов (индексы начинаются с 1), а в отрицательных автоматически появится единичная матрица a[i, -i] = 1"""

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1] + self.n
            return self.m[i, j]
        else:
            return self.m[index - 1]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1] + self.n
            self.m[i, j] = value
        else:
            self.m[index - 1] = value

    def __init__(self, nrows):
        self.n = nrows
        self.m = numpy.zeros((nrows, 2 * nrows + 1))
        
        for i in range(1, nrows + 1):
            self[i, -i] = 1


# Алгоритм начинается здесь
    
    def forwardGauss(self):
        """Прямой ход метода Гаусса"""
        for i in range(1, self.n + 1):

            ii = self[i, i]
            self[i] /= ii

            for i_ in range(i + 1, self.n + 1):
                i_i = self[i_, i]
                self[i_] -= self[i] * i_i
                
    def backwardGauss(self):
        """Обратный ход метода Гаусса"""
        for i in range(self.n, 0, -1):
            for i_ in range(i-1, 0, -1):
                i_i = self[i_, i]
                self[i_] -= self[i] * i_i

    
    def solveGauss(self):
        """Решает систему методом Гаусса, возвращает список найденных иксов. Предполагается, что есть диагональное преобладание."""
        self.forwardGauss()
        self.backwardGauss()
        return [self[i, 0] for i in range(1, self.n+1)]


In [248]:
class LinearSystem:
    """Сохраняет исходную матрицу в атрибуте source : Matrix, реализует метод Гаусса, Зейделя и умеет считать число обусловленности."""
    def __getitem__(self, index):
        return self.source.__getitem__(index)
    def __setitem__(self, index, value):
        self.source.__setitem__(index, value)

    def __init__(self, n):
        self.n = n
        self.source = Matrix(n)

    def solveGauss(self):
        """Решает систему методом Гаусса, возвращает список найденных иксов. Предполагается, что есть диагональное преобладание."""
        g = GaussMatrix(self.n)

        for i in range(1, self.n + 1):
            for j in range(0, self.n + 1):
                g[i, j] = self[i, j]
        
        ans = g.solveGauss()

        # Надо сохранить обратную матрицу, которая получилась как побочный продукт при решении методом Гаусса, чтобы потом считать число обусловленности
        self.inverse = Matrix(self.n)
        for i in range(self.n+1):
            for j in range(self.n+1):
                self.inverse[i, j] = g[i, -j]
        return ans





    def conditionNumber(self):
        """Число обусловленности по первой норме. Считает обратную матрицу по методу Гаусса, если она уже не посчитана."""
        if not hasattr(self, "inverse"):
            self.solveGauss()
        return self.source.norm() * self.inverse.norm()

    def lambdamin(self):
        """Минимальное по модулю собственное число. Считаем обратную матрицу по методу Гаусса, если она уже не посчитана."""
        if not hasattr(self, "inverse"):
            self.solveGauss()
        return 1 / self.inverse.lambdamax()

    def lambdamax(self):
        """Максимальное по модулю собственное число"""
        return self.source.lambdamax()







    def solveSeidel(self, max_residual):
        """Решает систему методом Зейделя, возвращает список найденных иксов. Предполагается, что метод сходится. Останавливается, когда невязка становится меньше max_residual"""
        ld = GaussMatrix(self.n)
        for i in range(1, self.n + 1):
            for j in range(1, i + 1):
                ld[i, j] = self.source[i, j]
        ld.forwardGauss()

        ld_inv = numpy.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(self.n):
                ld_inv[i, j] = ld.m[i, self.n - j - 1] # эквив ld[i', -j'], где i' = i + 1, j' = j + 1

        # A = numpy.zeros((self.n, self.n))
        # for i in range(self.n):
        #     for j in range(self.n):
        #         A[i, j] = self.source[i + 1, j + 1]
        A = self.source.m[:,1:]

        u = numpy.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(i + 1, self.n):
                u[i, j] = A[i, j]

        ld_inv_times_u = numpy.matmul(ld_inv, u)
        b = numpy.array([self.source[i, 0] for i in range(1, self.n+1)])
        ld_inv_times_b = ld_inv.dot(b)

        x = numpy.zeros(self.n)
        while numpy.max(abs(A.dot(x) - b)) > max_residual:
            x = - ld_inv_times_u.dot(x) + ld_inv_times_b

        return x.tolist()


Содержательный код закончился, дальше я его запускаю

In [249]:
def scaryMatrix(nrows):
    """Обобщение системы из условия на случай матриц произвольного размера.
        @returns LinearSystem"""
    m = LinearSystem(nrows)

    for i in range(2, nrows):
        m[i, i] = 10
        m[i, i-1] = m[i, i+1] = 1
    m[1, 1] = m[nrows, nrows] = 10
    m[1, 2] = 1

    for i in range(1, nrows+1):
        m[nrows, i] = 1
        
    for i in range(1, nrows+1):
        m[i, 0] = i
    return m

s = scaryMatrix(100)

In [250]:
ansGauss = s.solveGauss()

In [251]:
# Проверяем, что решение верно

s.source.dot(ansGauss) # - [i for i in range(1, 101)]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [252]:
ansSeidel = s.solveSeidel(1e-8)

In [253]:
# Проверяем, что решение верно

s.source.dot(ansSeidel) # - [i for i in range(1, 101)]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [254]:
# Число обусловленности
s.conditionNumber()

1020.127651091681

In [272]:
(s.lambdamax(), s.lambdamin())

(12.075227898404833, 0.899860853796566)

In [267]:
qwe = LinearSystem(3)
qwe.source.m = numpy.array([[0, 18, 6, 0],
                            [0, 6, 6, -7],
                            [0,0, -7, 18]])
(qwe.lambdamax(), qwe.lambdamin())

(22.9758020876225, 1.000000310067717)